In [51]:
import pandas as pd
from transformers import pipeline, AutoTokenizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
from torch.utils.data import Dataset, DataLoader

In [52]:
filepath='/content/sentimentdataset.csv'

In [53]:
# 1. Load the dataset
def load_dataset(file_path):
    dataset = pd.read_csv(file_path)
    print("Dataset loaded. First few rows:")
    print(dataset.head())
    return dataset

In [55]:
# 2. Preprocess the dataset (extract text and labels)
def preprocess_dataset(dataset):
    texts = dataset['Text'].tolist()
    labels = dataset['Sentiment'].apply(lambda x: 1 if x == 'positive' else 0).tolist()  # Encode positive as 1, negative as 0

    # Check dataset structure
    print(f"Sample texts: {texts[:2]}")
    print(f"Sample labels: {labels[:2]}")

    return texts, labels

In [56]:
# 3. Define emotion-to-polarity mapping
emotion_mapping = {
    'joy': 'positive',
    'surprise': 'positive',
    'trust': 'positive',
    'love': 'positive',
    'neutral': 'positive',

    'anger': 'negative',
    'fear': 'negative',
    'sadness': 'negative',
    'disgust': 'negative'
}

In [57]:
def map_emotion_to_polarity(emotion):
    return emotion_mapping.get(emotion, 'neutral')

In [45]:
def tokenize_function(examples, tokenizer):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

In [58]:
def load_emotion_model():
    tokenizer = AutoTokenizer.from_pretrained("j-hartmann/emotion-english-distilroberta-base")
    emotion_model = pipeline('text-classification', model='j-hartmann/emotion-english-distilroberta-base', return_all_scores=True)
    return emotion_model

In [68]:
def analyze_emotions_polarity(emotion_model, tweets):
    emotions = emotion_model(tweets)  # Get emotions for all tweets at once
    polarities = []

    for tweet_emotions in emotions:
        # Select the emotion with the highest score
        main_emotion = max(tweet_emotions, key=lambda x: x['score'])['label']
        polarity = map_emotion_to_polarity(main_emotion)
        polarities.append(1 if polarity == 'positive' else 0)

    return polarities

In [69]:
def evaluate_model(true_labels, predicted_labels):
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='binary')

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    return accuracy, precision, recall, f1

In [61]:
def evaluate_user(file_path):
    # Step 1: Load the dataset
    dataset = load_dataset(file_path)

    # Step 2: Preprocess the dataset
    texts, labels = preprocess_dataset(dataset)

    # Step 3: Load the pretrained emotion model
    emotion_model = load_emotion_model()

    # Step 4: Analyze emotions and map to positive/negative
    predicted_polarities = analyze_emotions_polarity(emotion_model, texts)

    # Step 5: Evaluate model performance
    evaluate_model(labels, predicted_polarities)

In [64]:
dataset = load_dataset(file_path)

Dataset loaded. First few rows:
   Unnamed: 0.1  Unnamed: 0  \
0             0           0   
1             1           1   
2             2           2   
3             3           3   
4             4           4   

                                                Text    Sentiment  \
0   Enjoying a beautiful day at the park!        ...   Positive     
1   Traffic was terrible this morning.           ...   Negative     
2   Just finished an amazing workout! 💪          ...   Positive     
3   Excited about the upcoming weekend getaway!  ...   Positive     
4   Trying out a new recipe for dinner tonight.  ...   Neutral      

             Timestamp            User     Platform  \
0  2023-01-15 12:30:00   User123          Twitter     
1  2023-01-15 08:45:00   CommuterX        Twitter     
2  2023-01-15 15:45:00   FitnessFan      Instagram    
3  2023-01-15 18:20:00   AdventureX       Facebook    
4  2023-01-15 19:55:00   ChefCook        Instagram    

                                   

In [65]:
texts, labels = preprocess_dataset(dataset)

Sample texts: [' Enjoying a beautiful day at the park!              ', ' Traffic was terrible this morning.                 ']
Sample labels: [0, 0]


In [66]:
emotion_model = load_emotion_model()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [70]:
predicted_polarities = analyze_emotions_polarity(emotion_model, texts)

In [ ]:
evaluate_model(labels, predicted_polarities)